In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [2]:
(train_images, train_labels), (test_images, test_labels) = keras.datasets.fashion_mnist.load_data()

train_images  = train_images / 255.0
train_images = np.expand_dims(train_images, axis=-1)
print(f"train_images.shape = {train_images.shape}, train_labels.shape = {train_labels.shape}")
test_images = test_images / 255.0
test_images = np.expand_dims(test_images, axis=-1)
print(f"test_images.shape = {test_images.shape}, test_labels.shape = {test_labels.shape}")

train_images.shape = (60000, 28, 28, 1), train_labels.shape = (60000,)
test_images.shape = (10000, 28, 28, 1), test_labels.shape = (10000,)


In [3]:
def build_model():
    model = keras.models.Sequential([
        keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(28, 28, 1)),
        keras.layers.MaxPooling2D(2, 2),
        keras.layers.Conv2D(64, (3, 3), activation='relu'),
        keras.layers.MaxPooling2D(2, 2),
        keras.layers.Flatten(),
        keras.layers.Dense(128, activation='relu'), 
        keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(
        optimizer=keras.optimizers.Adam(),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

## ModelCheckpoint

In [4]:
cb_checkpoint = keras.callbacks.ModelCheckpoint(
    'model_{epoch:02d}_{val_loss:.4f}.h5', 
    save_best_only=True,
    verbose=1
)

In [5]:
keras.backend.clear_session()

model = build_model()

history = model.fit(
    train_images,
    train_labels, 
    epochs=10,
    batch_size=64,
    validation_data=(test_images, test_labels),
    callbacks=[cb_checkpoint],
    verbose=2
)

Epoch 1/10

Epoch 00001: val_loss improved from inf to 0.34827, saving model to model_01_0.3483.h5
938/938 - 3s - loss: 0.4710 - accuracy: 0.8300 - val_loss: 0.3483 - val_accuracy: 0.8745
Epoch 2/10

Epoch 00002: val_loss improved from 0.34827 to 0.29981, saving model to model_02_0.2998.h5
938/938 - 3s - loss: 0.3112 - accuracy: 0.8872 - val_loss: 0.2998 - val_accuracy: 0.8939
Epoch 3/10

Epoch 00003: val_loss improved from 0.29981 to 0.28246, saving model to model_03_0.2825.h5
938/938 - 3s - loss: 0.2631 - accuracy: 0.9042 - val_loss: 0.2825 - val_accuracy: 0.8941
Epoch 4/10

Epoch 00004: val_loss improved from 0.28246 to 0.26089, saving model to model_04_0.2609.h5
938/938 - 3s - loss: 0.2331 - accuracy: 0.9138 - val_loss: 0.2609 - val_accuracy: 0.9059
Epoch 5/10

Epoch 00005: val_loss improved from 0.26089 to 0.25364, saving model to model_05_0.2536.h5
938/938 - 3s - loss: 0.2075 - accuracy: 0.9230 - val_loss: 0.2536 - val_accuracy: 0.9063
Epoch 6/10

Epoch 00006: val_loss did not im

## EarlyStopping

In [6]:
cb_early_stopping = keras.callbacks.EarlyStopping(
    patience=2, 
    restore_best_weights=True,
    verbose=1
)

In [7]:
keras.backend.clear_session()

model = build_model()

history = model.fit(
    train_images,
    train_labels, 
    epochs=20,
    batch_size=64,
    validation_data=(test_images, test_labels),
    callbacks=[cb_early_stopping],
    verbose=2
)

Epoch 1/20
938/938 - 3s - loss: 0.4692 - accuracy: 0.8320 - val_loss: 0.3583 - val_accuracy: 0.8706
Epoch 2/20
938/938 - 3s - loss: 0.3152 - accuracy: 0.8863 - val_loss: 0.3132 - val_accuracy: 0.8905
Epoch 3/20
938/938 - 3s - loss: 0.2698 - accuracy: 0.9016 - val_loss: 0.3077 - val_accuracy: 0.8897
Epoch 4/20
938/938 - 3s - loss: 0.2359 - accuracy: 0.9133 - val_loss: 0.3101 - val_accuracy: 0.8866
Epoch 5/20
938/938 - 3s - loss: 0.2155 - accuracy: 0.9204 - val_loss: 0.2713 - val_accuracy: 0.9037
Epoch 6/20
938/938 - 3s - loss: 0.1917 - accuracy: 0.9289 - val_loss: 0.2509 - val_accuracy: 0.9066
Epoch 7/20
938/938 - 3s - loss: 0.1732 - accuracy: 0.9342 - val_loss: 0.2733 - val_accuracy: 0.9064
Epoch 8/20
Restoring model weights from the end of the best epoch.
938/938 - 3s - loss: 0.1564 - accuracy: 0.9423 - val_loss: 0.2532 - val_accuracy: 0.9127
Epoch 00008: early stopping


## Custom callback

In [8]:
from datetime import datetime

class TimestampLoggerCallback(keras.callbacks.Callback):
    def __init__(self):
        super().__init__()

    def on_epoch_begin(self, batch, logs=None):
        print(f"on_epoch_begin(): {self.current_timestamp()}")

    def on_epoch_end(self, batch, logs=None):
        print(f"on_epoch_end(): {self.current_timestamp()}")

    @staticmethod
    def current_timestamp():
        return datetime.now().strftime('%Y/%m/%d %H:%M:%S')


In [9]:
cb_custom = TimestampLoggerCallback()

In [10]:
keras.backend.clear_session()

model = build_model()

history = model.fit(
    train_images,
    train_labels, 
    epochs=5,
    batch_size=64,
    validation_data=(test_images, test_labels),
    callbacks=[cb_custom],
    verbose=2
)

on_epoch_begin(): 2020/11/23 10:23:50
Epoch 1/5
on_epoch_end(): 2020/11/23 10:23:53
938/938 - 3s - loss: 0.4832 - accuracy: 0.8246 - val_loss: 0.3663 - val_accuracy: 0.8647
on_epoch_begin(): 2020/11/23 10:23:53
Epoch 2/5
on_epoch_end(): 2020/11/23 10:23:56
938/938 - 3s - loss: 0.3166 - accuracy: 0.8839 - val_loss: 0.3100 - val_accuracy: 0.8888
on_epoch_begin(): 2020/11/23 10:23:56
Epoch 3/5
on_epoch_end(): 2020/11/23 10:23:59
938/938 - 3s - loss: 0.2685 - accuracy: 0.9018 - val_loss: 0.2824 - val_accuracy: 0.8981
on_epoch_begin(): 2020/11/23 10:23:59
Epoch 4/5
on_epoch_end(): 2020/11/23 10:24:02
938/938 - 3s - loss: 0.2378 - accuracy: 0.9128 - val_loss: 0.2767 - val_accuracy: 0.8997
on_epoch_begin(): 2020/11/23 10:24:02
Epoch 5/5
on_epoch_end(): 2020/11/23 10:24:05
938/938 - 3s - loss: 0.2142 - accuracy: 0.9214 - val_loss: 0.2609 - val_accuracy: 0.9026
